In [1]:
%load_ext autoreload
%matplotlib inline

In [2]:
import os,sys,glob
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pickle 


/Users/bfildier/anaconda3/envs/pyLMD/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [3]:
## Add own library to path
workdir = os.getcwd()
repodir = workdir
projectname = 'aggregation-time-scale'
while os.path.basename(repodir) != projectname:
    repodir = os.path.dirname(repodir)
print('repo:',repodir)
thismodule = sys.modules[__name__]
## Own functions
moduledir = os.path.join(repodir,'functions')
sys.path.insert(0,moduledir)
print("Own modules available:", [os.path.splitext(os.path.basename(x))[0]
                                 for x in glob.glob(os.path.join(moduledir,'*.py'))])

repo: /Users/bfildier/Code/analyses/aggregation-time-scale
Own modules available: ['conditionalstats', 'moistdryedge']


In [4]:
# local input directory
inputdir = os.path.join(repodir,'input','irene')
figdir = os.path.join(repodir,'figures')
resultdir = os.path.join(repodir,'results')

## Load own libraries
from moistdryedge import *

## Graphical parameters
plt.style.use(os.path.join(matplotlib.get_configdir(),'stylelib/presentation.mplstyle'))

# Load data

In [5]:
simname = 'SAM6113_RCE_SST305d1p0r1'
print('simname:',simname)

# create output subdirectories
os.makedirs(os.path.join(figdir,simname),exist_ok=True)
os.makedirs(os.path.join(resultdir,simname),exist_ok=True)

# get sim-dependent variables
caseid = '_'.join(simname.split('_')[1:])
print('caseid:',caseid)
simdir = os.path.join(inputdir,simname+'.0')
inputfile2D = "%s_256.2Dcom_1.nc"%caseid

simname: SAM6113_RCE_SST305d1p0r1
caseid: RCE_SST305d1p0r1


In [6]:
print('load 2D data')
data2D = xr.open_dataset(os.path.join(simdir,inputfile2D))

load 2D data


# Calculate stats over time

In [7]:
vars4stats = ['PW','CRH','LHF','SHF','LWNT','LWNS']

## Edge 

In [8]:
print('Calculate edge and stats for boundary defined based on CRH and PW')

#- Get edge
# PW
edge_PW = EdgeOverTime()
edge_PW.compute(data2D.PW)
# CRH
edge_CRH = EdgeOverTime()
edge_CRH.compute(data2D.CRH)

#- Calculate stats
for varid in vars4stats:
    print(varid,end=', ')
    for edge in edge_PW,edge_CRH:
        
        # mean on edge
        edge.computeStatOnEdge(data2D[varid],varid)
        # std on edge
        edge.computeStatOnEdge(data2D[varid],varid,fname='std')
        # mean of gradient norm on edge
        edge.computeGradNormStatOnEdge(data2D[varid],varid)
        # std of gradient norm on edge
        edge.computeGradNormStatOnEdge(data2D[varid],varid,fname='std')

print()

Calculate edge and stats for boundary defined based on CRH and PW
PW, 

/Users/bfildier/anaconda3/envs/pyLMD/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/bfildier/anaconda3/envs/pyLMD/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/bfildier/anaconda3/envs/pyLMD/lib/python3.7/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/bfildier/anaconda3/envs/pyLMD/lib/python3.7/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/bfildier/anaconda3/envs/pyLMD/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


CRH, LHF, SHF, LWNT, LWNS, 


In [9]:
print("save edges defined from PW and CRH")

pickle.dump(edge_PW,open(os.path.join(resultdir,simname,"edge_PW.p"),"wb"))
pickle.dump(edge_CRH,open(os.path.join(resultdir,simname,"edge_CRH.p"),"wb"))

save edges defined from PW and CRH


In [10]:
def showErrorRange(ax,x,y,dy,**kwargs):
    
    ax.fill_between(x,y-dy,y+dy,**kwargs)

In [52]:
print("plot stats on the boundary: compare edges defined from CRH and PW")

x = data2D.time

for varid in vars4stats:

    # create figure 
    fig,ax0 = plt.subplots(ncols=1,figsize=(6,5))
    
    for refvarid in 'PW','CRH':

        edge = getattr(thismodule,"edge_%s"%refvarid)
        # Get data
        v_mean = getattr(edge,"%s_mean"%varid)
        v_std = getattr(edge,"%s_std"%varid)
        # plot
        showErrorRange(ax0,x,v_mean,v_std,alpha=0.2)
        ax0.plot(x,v_mean,label='%s-edge'%refvarid)

    # labels
    ax0.set_xlabel('Days')
    ax0.set_ylabel('%s (%s)'%(varid,data2D[varid].units))
    ax0.legend()

    # save and close figure
    plt.savefig(os.path.join(figdir,simname,"%s_onPWandCRHedge.pdf"%varid),bbox_inches='tight')
    plt.close()


plot stats on the boundary: compare edges defined from CRH and PW


In [ ]:
print("compute fraction area from thresholds based on PW and CRH")



## Conditional stats

## Tests